In [5]:
from langchain_community.tools import TavilySearchResults
from langchain_groq import ChatGroq
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


class State(TypedDict):
    value: str


def node_a(state: State) -> Command:
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "value": state["value"] + "A"
        }
    )


def node_b(state: State) -> Command:
    print("Node B")
    return Command(
        goto="node_c",
        update={
            "value": state["value"] + "B"
        }
    )


def node_c(state: State) -> Command:
    print("Node C")
    return Command(
        goto="node_d",
        update={
            "value": state["value"] + "C"
        }
    )


def node_d(state: State) -> Command:
    print("Node D")
    return Command(
        goto=END,
        update={
            "value": state["value"] + "D"
        }
    )


graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config = {
    "configurable" :  {
        "thread_id": "1",
    }
}

initialState = {
    "value": ""
}


first_result = app.invoke(
    initialState,
    config,
    stream_mode="updates")

first_result


Node A
Node B
Node C
Node D


[{'node_a': {'value': 'A'}},
 {'node_b': {'value': 'AB'}},
 {'node_c': {'value': 'ABC'}},
 {'node_d': {'value': 'ABCD'}}]